# Concept of a Double Tilted Rowland Spectrograph

In [ ]:
from copy import deepcopy
import warnings
import transforms3d
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
from x3d import x3d

import marxs
from marxs.visualization.x3d import plot_object, plot_rays
from marxs import optics, source, simulator
from marxs.design import rowland

In [ ]:
demo_rowland = rowland.RowlandTorus(500, 500)

demo_aperture = optics.CircleAperture(position=[1200, 0, 0],
                                      zoom=[1, 100, 100])
demo_mirror = optics.FlatStack(position=[1100, 0, 0], zoom=[20, 100, 100],
                               elements=[optics.PerfectLens,
                                         optics.RadialMirrorScatter],
                               keywords=[{'focallength': 1100},
                                         {'inplanescatter': 1e-3 * u.rad,
                                          'perpplanescatter': 1e-4 * u.rad}])

demo_gas_kwargs = {"rowland": demo_rowland, 'd_element': [25, 25],
                   "guess_distance": 900,
                   "radius": [20, 100],
                   "elem_class": marxs.optics.FlatGrating,
                   "elem_args": {'d': 1e-5, 'zoom': [1, 10, 10],
                                 'order_selector': marxs.optics.OrderSelector([-1, 0, 1])}
                  }

demo_gas = rowland.GratingArrayStructure(**demo_gas_kwargs)
demo_gas_sub1 = rowland.GratingArrayStructure(phi=[-0.5 + 0.5 * np.pi, .5 + 0.5 * np.pi], **demo_gas_kwargs)
demo_gas_sub2 = rowland.GratingArrayStructure(phi=[-0.5 + 1.5 * np.pi, .5 + 1.5 * np.pi], **demo_gas_kwargs)

demo_det_kwargs = {'d_element': [10.5, 10.5], 
                   'elem_class': optics.FlatDetector,
                   'elem_args': {'zoom': [1, 5, 5], 'pixsize': 0.01},
                   'guess_distance': 5.,
                   'id_col': 'CCD_ID'}
demo_det = rowland.RectangularGrid(rowland=demo_rowland, y_range=[-500, 500],
                                   **demo_det_kwargs)

detfp = optics.FlatDetector(pixsize=1., zoom=[1, 500, 500])
detfp.display = deepcopy(detfp.display)
detfp.display['opacity'] = 0.2

demo_det.elements[0].display['color'] = 'orange'

demo_onaxis_full = simulator.Sequence(elements=[demo_aperture, demo_mirror,
                                                demo_gas, demo_det, detfp])

demo_onaxis_sub = simulator.Sequence(elements=[demo_aperture, demo_mirror,
                                               demo_gas_sub1, demo_gas_sub2,
                                               demo_det, detfp])

In [ ]:
def run_photons(instrument):
    keeppos = simulator.KeepCol('pos')
    instrument.postprocess_steps = [keeppos]
    target = SkyCoord(30., 30., unit='deg')
    star = source.PointSource(coords=target, energy=.5 * u.keV, flux=1. / u.cm**2 / u.s)
    pointing = source.FixedPointing(coords=target)
    photons = star.generate_photons(5000 * u.s)
    photons = pointing(photons)
    photons = instrument(photons)
    ind = (photons['probability'] >= 0) & (photons['facet'] >= 0)
    posdat = keeppos.format_positions()[ind, :, :]
    return posdat, photons[ind]


In [ ]:
instrument = demo_onaxis_sub

In [ ]:
posdat, photons = run_photons(instrument)

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    out = plot_object(instrument)

out = plot_rays(posdat, scalar=photons['order'], scene=out)

out.children.append(x3d.Viewpoint(description='Overview ', orientation=(.5, .3, .3, -.3), viewAll=True))
out.children.append(x3d.Viewpoint(description='Camera', position=(-700, 50, 500), 
                                  orientation=(-0., -0.6, -0., 0.3)
                                  ))    
out

To do to make figures:

- switch off the big detector
- select better viewpoint for camera
- bigger CCDs or maybe continuous detector? Or bend CCDs? At least make ring have less angle so it's less obvious that it spreads out in the end. Or fix code the distributes them to actually place evenly on rig with constant gap size. Maybe that would be best.
- Transparancy for some objects (e.g. aperture): https://doc.x3dom.org/author/Shape/Material.html
- make plots of how the spot actually looks like on a detector
- Add Rowland torus to the plot
- Next two plots:
    - Double rowland
    - Double rowland with channel offset (chip gaps!) - select parameters to that simple double rowland has example wavelength fall in (or near) chip gap